In [2]:
import numpy as np 
import pandas as pd
import json
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
data=pd.read_csv('movies.csv')

In [4]:
data.head(2)

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski


In [5]:
data.shape

(4803, 24)

In [6]:
data.describe()

,index,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4803.000000,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2401.000000,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,1386.651002,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,1200.500000,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,2401.000000,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,3601.500000,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,4802.000000,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [7]:
data.isna().sum()

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [8]:
data['production_companies'][0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [9]:
def deleate_features(df):
    df.drop(columns=['budget','homepage','id','status'],axis=1,inplace=True)
    return df

In [10]:
df=deleate_features(data)

In [11]:
df.shape

(4803, 20)

In [12]:
df.tail(4)

,index,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast,crew,director
4799,4799,Comedy Romance,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,Comedy Drama Romance TV Movie,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""id"": 3958}...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2013-10-13,0,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,NaN,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,[],"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-05-03,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",A New Yorker in Shanghai,Shanghai Calling,5.7,7,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,"[{'name': 'Daniel Hsia', 'gender': 2, 'departm...",Daniel Hsia
4802,4802,Documentary,obsession camcorder crush dream girl,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,"[{""name"": ""rusty bear entertainment"", ""id"": 87...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2005-08-05,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",NaN,My Date with Drew,6.3,16,Drew Barrymore Brian Herzlinger Corey Feldman ...,"[{'name': 'Clark Peterson', 'gender': 2, 'depa...",Brian Herzlinger


In [13]:
df['genres'].str.split(' ')

0       [Action, Adventure, Fantasy, Science, Fiction]
1                         [Adventure, Fantasy, Action]
2                           [Action, Adventure, Crime]
3                     [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science, Fiction]
                             ...                      
4798                         [Action, Crime, Thriller]
4799                                 [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV, Movie]
4801                                               NaN
4802                                     [Documentary]
Name: genres, Length: 4803, dtype: object

In [14]:
# Application de la méthode str.get_dummies() pour créer des colonnes binaires pour chaque genre
#Genre = df['genres'].str.get_dummies(sep=' ')

# Concaténation du DataFrame original avec les colonnes binaires
#df = pd.concat([df, Genre], axis=1)


In [15]:
df.columns

Index(['index', 'genres', 'keywords', 'original_language', 'original_title',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'tagline', 'title', 'vote_average', 'vote_count',
       'cast', 'crew', 'director'],
      dtype='object')

In [16]:
#df.drop(columns=['genres'],inplace=True)

In [17]:
df.iloc[:4,:18]

,index,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count,cast
0,0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...
1,1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...
2,2,Action Adventure Crime,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...
3,3,Action Crime Drama Thriller,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...


In [18]:
df['production_companies'][0]

'[{"name": "Ingenious Film Partners", "id": 289}, {"name": "Twentieth Century Fox Film Corporation", "id": 306}, {"name": "Dune Entertainment", "id": 444}, {"name": "Lightstorm Entertainment", "id": 574}]'

In [19]:
## production_companies

for i in range(len(df)):
    liste_dict = json.loads(df['production_companies'][i])
    liste_ids = ' '

    for d in liste_dict:
        liste_ids=liste_ids+ d['name'] + ', '

    df['production_companies'][i]=liste_ids

C:\Users\Xps\AppData\Local\Temp\ipykernel_20408\453267029.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['production_companies'][i]=liste_ids


In [20]:
df['production_companies']

0        Ingenious Film Partners, Twentieth Century Fo...
1        Walt Disney Pictures, Jerry Bruckheimer Films...
2                        Columbia Pictures, Danjaq, B24, 
3        Legendary Pictures, Warner Bros., DC Entertai...
4                                  Walt Disney Pictures, 
                              ...                        
4798                                  Columbia Pictures, 
4799                                                     
4800     Front Street Pictures, Muse Entertainment Ent...
4801                                                     
4802         rusty bear entertainment, lucky crow films, 
Name: production_companies, Length: 4803, dtype: object

In [21]:
## production_countries

for i in range(len(df)):
    liste_dict = json.loads(df['production_countries'][i])
    liste_ids = ' '

    for d in liste_dict:
        liste_ids=liste_ids+ d['name'] + ', '

    df['production_countries'][i]=liste_ids

C:\Users\Xps\AppData\Local\Temp\ipykernel_20408\2730624988.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['production_countries'][i]=liste_ids


In [22]:
df['production_countries']

0        United States of America, United Kingdom, 
1                        United States of America, 
2        United Kingdom, United States of America, 
3                        United States of America, 
4                        United States of America, 
                           ...                     
4798             Mexico, United States of America, 
4799                                               
4800                     United States of America, 
4801              United States of America, China, 
4802                     United States of America, 
Name: production_countries, Length: 4803, dtype: object

In [23]:
## spoken_languages

for i in range(len(df)):
    liste_dict = json.loads(df['spoken_languages'][i])
    liste_ids = ' '

    for d in liste_dict:
        liste_ids=liste_ids+ d['name'] + ', '

    df['spoken_languages'][i]=liste_ids

C:\Users\Xps\AppData\Local\Temp\ipykernel_20408\620485191.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['spoken_languages'][i]=liste_ids


In [24]:
df['spoken_languages']

0                                     English, Español, 
1                                              English, 
2        Français, English, Español, Italiano, Deutsch, 
3                                              English, 
4                                              English, 
                              ...                       
4798                                           Español, 
4799                                                    
4800                                           English, 
4801                                           English, 
4802                                           English, 
Name: spoken_languages, Length: 4803, dtype: object

In [25]:
df.iloc[:3,:17]

,index,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,tagline,title,vote_average,vote_count
0,0,Action Adventure Fantasy Science Fiction,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"Ingenious Film Partners, Twentieth Century Fo...","United States of America, United Kingdom,",2009-12-10,2787965087,162.0,"English, Español,",Enter the World of Pandora.,Avatar,7.2,11800
1,1,Adventure Fantasy Action,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"Walt Disney Pictures, Jerry Bruckheimer Films...","United States of America,",2007-05-19,961000000,169.0,"English,","At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,2,Action Adventure Crime,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"Columbia Pictures, Danjaq, B24,","United Kingdom, United States of America,",2015-10-26,880674609,148.0,"Français, English, Español, Italiano, Deutsch,",A Plan No One Escapes,Spectre,6.3,4466


In [26]:
df['year'] = pd.to_datetime(df['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [30]:
selected_features = ['year','genres','keywords','overview','production_companies','production_countries','spoken_languages','tagline','cast','director']

for feature in selected_features:
    df[feature] = df[feature].fillna('')

In [31]:
df['descreption']=df['year']+', '+df['genres']+', '+df['keywords']+', '+df['overview']+', '+df['production_companies']+', '+df['production_countries']+', '+df['spoken_languages']+', '+df['tagline']+', '+df['cast']+', '+df['director']

In [32]:
df['descreption']

0       2009, Action Adventure Fantasy Science Fiction...
1       2007, Adventure Fantasy Action, ocean drug abu...
2       2015, Action Adventure Crime, spy based on nov...
3       2012, Action Crime Drama Thriller, dc comics c...
4       2012, Action Adventure Science Fiction, based ...
                              ...                        
4798    1992, Action Crime Thriller, united states\u20...
4799    2011, Comedy Romance, , A newlywed couple's ho...
4800    2013, Comedy Drama Romance TV Movie, date love...
4801    2012, , , When ambitious New York attorney Sam...
4802    2005, Documentary, obsession camcorder crush d...
Name: descreption, Length: 4803, dtype: object

In [33]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(df['descreption'])
print(feature_vectors)

  (0, 4659)	0.1226419176534273
  (0, 15385)	0.07835509789977688
  (0, 25092)	0.13950674133788254
  (0, 19292)	0.11435638599907708
  (0, 16895)	0.1595077705274063
  (0, 28035)	0.10770606190008504
  (0, 32103)	0.14022452625453596
  (0, 26880)	0.147927799719674
  (0, 25574)	0.15639299466941312
  (0, 33128)	0.14022452625453596
  (0, 32673)	0.1698140360655723
  (0, 25618)	0.10282875124471316
  (0, 32648)	0.06724446418601962
  (0, 9795)	0.13501944202048807
  (0, 9984)	0.08429081539517151
  (0, 9736)	0.024913322247510958
  (0, 16366)	0.06948683939496114
  (0, 1299)	0.027762320419720297
  (0, 21088)	0.048343219647145816
  (0, 27941)	0.02782112097821018
  (0, 30999)	0.05267701927396086
  (0, 17405)	0.17927198378236986
  (0, 9806)	0.10842715117900394
  (0, 9073)	0.11584579623836229
  (0, 6712)	0.08017086491705225
  :	:
  (4802, 13608)	0.07867389170108958
  (4802, 13392)	0.10311307480916906
  (4802, 32261)	0.03531236070389809
  (4802, 13809)	0.028298003944680385
  (4802, 20569)	0.0398619907156153

In [34]:
similarity = cosine_similarity(feature_vectors)
similarity

array([[1.        , 0.05023907, 0.04625616, ..., 0.03412179, 0.03258527,
        0.01368053],
       [0.05023907, 1.        , 0.04564362, ..., 0.05141091, 0.03301457,
        0.02846826],
       [0.04625616, 0.04564362, 1.        , ..., 0.02996872, 0.04843745,
        0.01552837],
       ...,
       [0.03412179, 0.05141091, 0.02996872, ..., 1.        , 0.04415534,
        0.03824767],
       [0.03258527, 0.03301457, 0.04843745, ..., 0.04415534, 1.        ,
        0.03201614],
       [0.01368053, 0.02846826, 0.01552837, ..., 0.03824767, 0.03201614,
        1.        ]])

In [35]:
# getting the movie name from the user

movie_name = input(' Enter your favourite movie name : ')
# creating a list with all the movie names given in the dataset

list_of_all_titles = df['original_title'].tolist()
#print(list_of_all_titles)

 Enter your favourite movie name :  inception


In [36]:

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['Inception', 'Minions', 'Lincoln']


In [38]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = df['original_title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.original_title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
	index = movie[0]
	title_from_index = df[df.index==index]['original_title'].values[0]
	if (i<30):
		print(i, '.',title_from_index)
		i+=1

 Enter your favourite movie name :  inception


Movies suggested for you : 

1 . Inception
2 . The Dark Knight
3 . Mad Max: Fury Road
4 . The Dark Knight Rises
5 . The Town
6 . Interstellar
7 . Batman Begins
8 . Looper
9 . Kiss Kiss Bang Bang
10 . The Prestige
11 . Hesher
12 . The Helix... Loaded
13 . The Last Samurai
14 . (500) Days of Summer
15 . Tinker Tailor Soldier Spy
16 . Body of Lies
17 . Man of Steel
18 . Don Jon
19 . Letters from Iwo Jima
20 . The Walk
21 . G.I. Joe: The Rise of Cobra
22 . Edge of Darkness
23 . Hudson Hawk
24 . The Exorcist
25 . Sugar Town
26 . The Revenant
27 . Harry Potter and the Order of the Phoenix
28 . Cypher
29 . Highlander: Endgame


#### Save the model

In [39]:
filename="trained_model.sav"
pickle.dump(vectorizer, open(filename,'wb'))

In [40]:
# loading the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [41]:
input_data = "Spiderman"

list_of_all_titles = df['original_title'].tolist()

find_close_match = difflib.get_close_matches(input_data, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = df[df.original_title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True) 

print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
	index = movie[0]
	title_from_index = df[df.index==index]['original_title'].values[0]
	if (i<30):
		print(i, '.',title_from_index)
		i+=1

Movies suggested for you : 

1 . Spider-Man
2 . Spider-Man 3
3 . Spider-Man 2
4 . Arachnophobia
5 . The Amazing Spider-Man 2
6 . The Amazing Spider-Man
7 . Charlotte's Web
8 . Oz: The Great and Powerful
9 . xXx: State of the Union
10 . Shadow of the Vampire
11 . 21 Jump Street
12 . Brothers
13 . The Good German
14 . The Ice Storm
15 . Gremlins 2: The New Batch
16 . Seabiscuit
17 . Midnight Special
18 . The Covenant
19 . The Notebook
20 . X-Men
21 . Speed 2: Cruise Control
22 . Home for the Holidays
23 . Get Over It
24 . The New Guy
25 . X-Men: Apocalypse
26 . X2
27 . Bachelorette
28 . Guardians of the Galaxy
29 . Ghost Rider: Spirit of Vengeance
